# Going From Staregies A&B To A Portfolio

So far, I have implemented two startegies using momentum and news analytics on Quantopian along with their backtests. The universe of assets is considered to be Q1500US - the 1500 most liquid equities in the US stock exchange. In this notebook I will attempt to take the best performing equities from those startegies for my portfolio and size them appropriately

## Background Work

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from matplotlib import pyplot as plt
from IPython.display import display

pd.options.display.max_rows = 5

### Defining All Equities

In [2]:
equities_strategyA = [
    'ARNC',
    'ABT',
    'ADSK',
    'ADBE',
    'ADI',
    'AEP',
    'AES',
    'AFL',
    'AMD',
    'ADP'
]

equities_strategyB = [
    'ENDP',
    'MDCO',
    'WW',
    'STMP',
    'INFN',
    'SEMG',
    'MIK',
    'RETA',
    'CLDR',
    'HOME',
]

### Querying Data From Yahoo Finance

In [3]:
end = datetime.now().strftime("%Y-%m-%d")
start = datetime.now() - relativedelta(years=1)


data_strategyA = yf.download(equities_strategyA,
                             start,
                             end)['Close']
data_strategyB = yf.download(equities_strategyB,
                             start,
                             end)['Close']

print("\nFields:", data_strategyA.columns.get_level_values(0).unique())

display(data_strategyA)
display(data_strategyB)

data_strategyA.plot(figsize=(15,5)); plt.grid(); plt.title('StrategyA')
data_strategyB.plot(figsize=(15,5)); plt.grid(); plt.title('StrategyB')

[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  10 of 10 completed

Fields: Index(['ABT', 'ADBE', 'ADI', 'ADP', 'ADSK', 'AEP', 'AES', 'AFL', 'AMD',
       'ARNC'],
      dtype='object')


,ABT,ADBE,ADI,ADP,ADSK,AEP,AES,AFL,AMD,ARNC
Date,,,,,,,,,,
2018-12-10,70.919998,244.089996,88.480003,139.729996,135.020004,79.360001,15.770000,43.040001,19.990000,19.889999
2018-12-11,71.839996,245.339996,88.639999,139.990005,133.869995,79.760002,15.740000,42.639999,19.980000,20.139999
...,...,...,...,...,...,...,...,...,...,...
2019-12-05,85.239998,303.029999,113.430000,168.360001,176.630005,92.459999,18.809999,52.720001,39.619999,30.480000
2019-12-06,85.480003,306.230011,115.889999,169.979996,179.320007,92.389999,18.719999,52.990002,39.630001,31.020000


,CLDR,ENDP,HOME,INFN,MDCO,MIK,RETA,SEMG,STMP,WW
Date,,,,,,,,,,
2018-12-10,12.73,11.35,20.280001,4.35,20.790001,15.55,59.560001,15.40,172.809998,49.169998
2018-12-11,12.32,11.21,20.980000,4.23,20.700001,15.39,59.310001,15.49,168.300003,49.180000
...,...,...,...,...,...,...,...,...,...,...
2019-12-05,10.01,4.82,5.480000,5.99,83.900002,6.08,199.559998,15.02,84.150002,43.180000
2019-12-06,10.88,4.84,5.670000,6.12,84.209999,6.69,199.320007,NaN,82.910004,39.919998


### Volatility Calculation

In [4]:
volatilityA = data_strategyA.rolling(22).std().dropna(how='all')
volatilityB = data_strategyB.rolling(22).std().dropna(how='all')

display(volatilityA)
display(volatilityB)

volatilityA.plot(figsize=(15,5)); plt.grid(); plt.title('StrategyA')
volatilityB.plot(figsize=(15,5)); plt.grid(); plt.title('StrategyB')

,ABT,ADBE,ADI,ADP,ADSK,AEP,AES,AFL,AMD,ARNC
Date,,,,,,,,,,
2019-01-10,1.874416,11.363674,2.680921,4.630670,5.411718,2.587550,0.635278,1.055612,1.270815,1.304824
2019-01-11,1.852577,11.003731,2.783436,4.255053,5.505079,2.490537,0.614413,1.045209,1.283486,1.295207
...,...,...,...,...,...,...,...,...,...,...
2019-12-05,0.848990,6.619794,1.664942,4.139426,9.736732,1.134081,0.514049,0.826628,1.607494,0.750735
2019-12-06,0.762799,6.478937,1.812388,3.395743,9.739645,1.184443,0.440987,0.824129,1.533832,0.742263


,CLDR,ENDP,HOME,INFN,MDCO,MIK,RETA,SEMG,STMP,WW
Date,,,,,,,,,,
2019-01-10,0.713747,1.392379,1.673771,0.232535,1.304520,1.036242,7.408383,1.116323,8.457618,4.815420
2019-01-11,0.653089,1.292187,1.774006,0.229449,1.257225,1.063560,7.972593,1.167889,8.404203,4.852337
...,...,...,...,...,...,...,...,...,...,...
2019-12-05,0.545134,0.367575,0.711171,0.502197,13.919361,0.853469,8.269015,0.449514,2.520704,4.089442
2019-12-06,0.644658,0.369625,0.914251,0.500273,13.968594,0.875605,7.907760,NaN,2.644413,4.076698


### Getting Asset Correlation (from Close Price)

In [5]:
corrA = data_strategyA.corr()
corrB = data_strategyB.corr()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(corrA)
    display(corrB)

,ABT,ADBE,ADI,ADP,ADSK,AEP,AES,AFL,AMD,ARNC
ABT,1.000000,0.901712,0.837624,0.888392,0.449050,0.874758,0.330685,0.842293,0.857334,0.788337
ADBE,0.901712,1.000000,0.846329,0.911186,0.701446,0.788919,0.438247,0.872724,0.880627,0.758374
ADI,0.837624,0.846329,1.000000,0.850205,0.677454,0.725152,0.529465,0.732934,0.740112,0.586838
ADP,0.888392,0.911186,0.850205,1.000000,0.659046,0.855015,0.439591,0.826331,0.867276,0.715948
ADSK,0.449050,0.701446,0.677454,0.659046,1.000000,0.339354,0.682040,0.594072,0.532687,0.260117
AEP,0.874758,0.788919,0.725152,0.855015,0.339354,1.000000,0.213076,0.819603,0.838596,0.853209
AES,0.330685,0.438247,0.529465,0.439591,0.682040,0.213076,1.000000,0.400156,0.460041,0.202555
AFL,0.842293,0.872724,0.732934,0.826331,0.594072,0.819603,0.400156,1.000000,0.837991,0.749286
AMD,0.857334,0.880627,0.740112,0.867276,0.532687,0.838596,0.460041,0.837991,1.000000,0.907019
ARNC,0.788337,0.758374,0.586838,0.715948,0.260117,0.853209,0.202555,0.749286,0.907019,1.000000


,CLDR,ENDP,HOME,INFN,MDCO,MIK,RETA,SEMG,STMP,WW
CLDR,1.000000,0.879738,0.871862,0.300338,-0.387610,0.820828,-0.105326,0.621494,0.710931,0.137341
ENDP,0.879738,1.000000,0.908869,-0.046965,-0.613240,0.912253,-0.271544,0.564550,0.733442,-0.009653
HOME,0.871862,0.908869,1.000000,-0.098390,-0.625829,0.853061,-0.342683,0.472129,0.538457,-0.210472
INFN,0.300338,-0.046965,-0.098390,1.000000,0.562887,-0.047820,0.583013,0.366268,0.238549,0.570438
MDCO,-0.387610,-0.613240,-0.625829,0.562887,1.000000,-0.614218,0.794119,0.000573,-0.394978,0.366885
MIK,0.820828,0.912253,0.853061,-0.047820,-0.614218,1.000000,-0.300405,0.662322,0.745071,0.091868
RETA,-0.105326,-0.271544,-0.342683,0.583013,0.794119,-0.300405,1.000000,0.273716,-0.144868,0.320149
SEMG,0.621494,0.564550,0.472129,0.366268,0.000573,0.662322,0.273716,1.000000,0.574752,0.369599
STMP,0.710931,0.733442,0.538457,0.238549,-0.394978,0.745071,-0.144868,0.574752,1.000000,0.493017
WW,0.137341,-0.009653,-0.210472,0.570438,0.366885,0.091868,0.320149,0.369599,0.493017,1.000000


## Using Calculations Above For Position Sizing

### Startegy A

In [6]:
volatilityMeanAdjusted = volatilityA.mean()/data_strategyA.mean()
volatilityMeanAdjusted = (volatilityMeanAdjusted)/volatilityMeanAdjusted.sum()


corrAdjusted = corrA.sum(axis=1) - 1
corrAdjusted = (corrAdjusted-corrAdjusted.min())/corrAdjusted.sum()

position_weightsA = 1/(volatilityMeanAdjusted+corrAdjusted)
position_weightsA = position_weightsA/position_weightsA.sum()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(position_weightsA*100)

ABT     10.792833
ADBE     9.053075
ADI      7.949187
ADP     10.597364
ADSK     8.901444
AEP     13.653395
AES     13.361640
AFL     11.731794
AMD      5.571342
ARNC     8.387926
dtype: float64


### Streategy B

In [7]:
volatilityMeanAdjusted = volatilityB.mean()/data_strategyB.mean()
volatilityMeanAdjusted = (volatilityMeanAdjusted)/volatilityMeanAdjusted.sum()


corrAdjusted = corrB.sum(axis=1) - 1
corrAdjusted = (corrAdjusted-corrAdjusted.min())/corrAdjusted.sum()

position_weightsB = 1/(volatilityMeanAdjusted+corrAdjusted)
position_weightsB = position_weightsB/position_weightsB.sum()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(position_weightsB*100)

CLDR     7.213448
ENDP     7.254855
HOME     8.044499
INFN     9.327146
MDCO    26.203660
MIK      7.806313
RETA    11.206088
SEMG     7.338191
STMP     6.373316
WW       9.232484
dtype: float64

## Allocating Risk Based On Weights Calulated

So far we have weights (that sum up to 1) for each strategy. How do we decide how much capital to put into each strategy? Here I will allocate 1/2 of the risk into each strategy and size the entire portfolio accordingly. I will be working with the assumption that the max drawdown for both the strategies combined is 10M. Based on a 50% risk allocation, this will mean I need to size each strategy's positions such the drawdown for the strategy does not exceed 5M.

### Strategy A

In [8]:
df_A = pd.concat([data_strategyA.iloc[-1, :], position_weightsA, volatilityA.mean()],
          axis=1)
df_A.columns = ['Price Today', 'Weight', 'Avg Volatility']
df_A['-2std'] = -2 * df_A['Avg Volatility']
df_A['Risk'] = 5e5 * df_A['Weight']
df_A['Shares'] = -(df_A['Risk']/df_A['-2std']).round()
df_A['Value Today'] = df_A['Shares'] * df_A['Price Today']

print('Total Dollars Invested Today: ', df_A['Value Today'].sum())


with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_A)

Total Dollars Invested Today:  12360871.165575027


,Price Today,Weight,Avg Volatility,-2std,Risk,Shares,Value Today
ABT,85.480003,0.107928,1.548699,-3.097398,53964.163589,17422.0,1.489233e+06
ADBE,306.230011,0.090531,6.663770,-13.327540,45265.376664,3396.0,1.039957e+06
ADI,115.889999,0.079492,3.453110,-6.906221,39745.933208,5755.0,6.669469e+05
ADP,169.979996,0.105974,2.994862,-5.989724,52986.818725,8846.0,1.503643e+06
ADSK,179.320007,0.089014,5.402083,-10.804165,44507.221062,4119.0,7.386191e+05
AEP,92.389999,0.136534,1.259923,-2.519846,68266.976847,27092.0,2.503030e+06
AES,18.719999,0.133616,0.443786,-0.887572,66808.198777,75271.0,1.409073e+06
AFL,52.990002,0.117318,0.877916,-1.755832,58658.972222,33408.0,1.770290e+06
AMD,39.630001,0.055713,1.412658,-2.825316,27856.708838,9860.0,3.907518e+05
ARNC,31.020000,0.083879,0.765890,-1.531779,41939.630068,27380.0,8.493276e+05


### Strategy B

In [9]:
df_B = pd.concat([data_strategyB.iloc[-1, :], position_weightsB, volatilityB.mean()],
          axis=1)
df_B.columns = ['Price Today', 'Weight', 'Avg Volatility']
df_B['-2std'] = -2 * df_B['Avg Volatility']
df_B['Risk'] = 5e5 * df_B['Weight']
df_B['Shares'] = -(df_B['Risk']/df_B['-2std']).round()
df_B['Value Today'] = df_B['Shares'] * df_B['Price Today']

print('Total Dollars Invested Today: ', df_B['Value Today'].sum())


with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_B)

Total Dollars Invested Today:  4916441.324528694


,Price Today,Weight,Avg Volatility,-2std,Risk,Shares,Value Today
CLDR,10.880000,0.072134,0.644861,-1.289723,36067.238173,27965.0,3.042592e+05
ENDP,4.840000,0.072549,0.553622,-1.107245,36274.275586,32761.0,1.585632e+05
HOME,5.670000,0.080445,1.323379,-2.646759,40222.497288,15197.0,8.616699e+04
INFN,6.120000,0.093271,0.284421,-0.568841,46635.730914,81984.0,5.017421e+05
MDCO,84.209999,0.262037,2.226840,-4.453679,131018.300033,29418.0,2.477290e+06
MIK,6.690000,0.078063,0.768592,-1.537185,39031.565045,25392.0,1.698725e+05
RETA,199.320007,0.112061,8.630593,-17.261187,56030.439038,3246.0,6.469927e+05
SEMG,NaN,0.073382,0.864447,-1.728893,36690.952988,21222.0,NaN
STMP,82.910004,0.063733,9.533070,-19.066140,31866.578859,1671.0,1.385426e+05
WW,39.919998,0.092325,2.127797,-4.255593,46162.422076,10847.0,4.330122e+05


## Portfolio (Optimized) 

In [10]:
portfolio = pd.concat([df_A[['Price Today', 'Shares', 'Value Today']], 
         df_B[['Price Today', 'Shares', 'Value Today']]],
         axis=0)
portfolio.columns = ['Price Today', 'Optimal Shares', 'Value Today']

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(portfolio)

,Price Today,Optimal Shares,Value Today
ABT,85.480003,17422.0,1.489233e+06
ADBE,306.230011,3396.0,1.039957e+06
ADI,115.889999,5755.0,6.669469e+05
ADP,169.979996,8846.0,1.503643e+06
ADSK,179.320007,4119.0,7.386191e+05
AEP,92.389999,27092.0,2.503030e+06
AES,18.719999,75271.0,1.409073e+06
AFL,52.990002,33408.0,1.770290e+06
AMD,39.630001,9860.0,3.907518e+05
ARNC,31.020000,27380.0,8.493276e+05


In [12]:
portfolio['Value Today'].sum()/1e6
portfolio.to_pickle('Optimal Portfolio.pkl')

## What About the Current Holdings?

At this point we have determined optimal position sizes based on the risk allocation from weights above. However, at the time, I have already submitted multiple trades. In this section I will investigate wheather the worst case, for the current risk allocation, takes me less than 1M.

In [13]:
current_B = pd.DataFrame([95480,
8700,
12880,
5400,
79540,
27880,
52900,
2180,
52840,
51780],
                        index = equities_strategyB,
                        columns = ['Shares Holding'])

current_A = pd.DataFrame([15860,
6160,
2520,
2720,
3620,
3620,
21800,
6760,
14840,
2600], 
                        index = equities_strategyA,
                        columns = ['Shares Holding'])

current_A['Risk'] = df_A['-2std']*current_A['Shares Holding']
current_B['Risk'] = df_B['-2std']*current_B['Shares Holding']

In [14]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(current_A)
    display(current_B)
    
print('Worst Case Drawdown:', current_A['Risk'].sum() + current_B['Risk'].sum())

,Shares Holding,Risk
ARNC,15860,-24294.018425
ABT,6160,-19079.974410
ADSK,2520,-27226.496870
ADBE,2720,-36250.907630
ADI,3620,-25000.518416
AEP,3620,-9121.842549
AES,21800,-19349.079508
AFL,6760,-11869.423253
AMD,14840,-41927.687198
ADP,2600,-15573.283448


,Shares Holding,Risk
ENDP,95480,-105719.748231
MDCO,8700,-38747.007324
WW,12880,-54812.044073
STMP,5400,-102957.153918
INFN,79540,-45245.630461
SEMG,27880,-48201.541652
MIK,52900,-81317.064959
RETA,2180,-37629.386712
CLDR,52840,-68148.944374
HOME,51780,-137049.156517


Worst Case Drawdown: -949520.9099288934


I'm barely meeting the criteria - I have my current (suboptimally weighted) portfolio structure such that the worst case drawdown is less than -1M. So for now I will keep my positions.

In [16]:
pd.concat([current_A, current_B], axis=0).to_pickle('Current Portfolio.pkl')